# Developing an Information Retrieval System with Spelling Correction and Wildcard Queries

## AmirAli Amini - 610399102

#### HW2



# توضیحات مسئله و چالش ها و بهبود ها

بزرگ ترین چالش این مسئله ساخت درخت ترای بود که با ساختن دو کلاس یکی برای هر نود و یکی برای کل درخت انجام شد

در این سوال تمام جایگشت های دوری یک رشته را با $ در انتهای آن به درست اضافه میکنم که بتوان طبق الگوریتم گفته شده در کتاب وایدکارت ها را بدست اورد؛


## کتابخانه ها 

###  from nltk import word_tokenize :
از این کتابخانه برای تکنایز کردن داده ها به این دلیل که توکنایز کردن دیتا سریع تر میشه استفاده کردم

###  from nltk.corpus import stopwords :
از این کتابخانه برای دریافت استاپینگ ورد های زبان انگلیسی استفاده کردم

###  import string:
از این کتابخانه برای دریافت پانچویشن های زبان انگلیسی استفاده کردم

###  import numpy as np:
از این کتابخانه برای جمع یک عدد با تمام اعضای یک آرایه استفاده کردم

###  import copy:
از این کتابخانه برای دیپ کپی کردن ارایه استفاده کردم

In [1275]:
from nltk.tokenize import word_tokenize


from nltk.corpus import stopwords # a library to tokenize input texts

import nltk


nltk.download('punkt')
nltk.download('stopwords') # stopping word in English language

import string # using to remove punctuation

import numpy as np

import copy

import os

import pickle 
import shutil


[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>
[nltk_data] Error loading stopwords: <urlopen error [Errno 61]
[nltk_data]     Connection refused>


In [1276]:

class searchEngine:
    def __init__(self , debug = False,tempDirectory="postingList") -> None: # constructor of class
        self.debug = debug
        # self.subPostingLists=[
        self.codedPostingLit=[] # gama coding posting list
        self.tempDirectory=tempDirectory
        self.stop = set(stopwords.words('english') + list(string.punctuation)) # all extra expression which should ignore

        # structure of postingList : list of {word : nameOfWord , docs :gamaCode of including docs}


    # binary search to find a word in posting list
    def searchPostingList(self, word,postingList):
        s= 0 
        e = len(postingList)
        if e <=0 :
            return 0
        e-=1
        while (1):
            if (e-s < 2):
                if (postingList[e]["word"] < word):
                    return e+1
                if (postingList[e]["word"] == word):
                    return e
                if (postingList[s]["word"] >= word):
                    return s

                return e
            mid = (s+e)/2
            mid = int(mid)
            if (word<postingList[mid]["word"]):
                e=mid
            elif (word> postingList[mid]["word"]):
                s = mid
            else :
                return mid
            
        # {word:str, indexes:list(int)}
    # binary search to find a word in each dictionary



    def addToPostingList(self, tokenizedText: list[str],docIndex:int,postingList=[]): # add tokenized word in posting list 
        for i in range(len(tokenizedText)):
            word = tokenizedText[i]
            index = self.searchPostingList(word,postingList=postingList) # find index of word in posting list
            if (len(postingList)>index): # check if index is not larger than posting list (if word is bigger that all words, search function returns len(postingList)+1)
                if (postingList[index]["word"] == word): # check if index is the index of the word
                    if (postingList[index]["docs"][-1]["doc"] == docIndex): # if we have already added the document index 
                        postingList[index]["docs"][-1]["indexes"].append(i) # as we read tokens in order of their index, we need to add token in end of the list

                    else:
                        postingList[index]["docs"].append({"doc":docIndex,"indexes":[i] }) # if we have not already added the document and dou to the fact that they are read in order of their index, we can easily add append new one in end of the list 

                else :
                    postingList[index:index]= [({"word":word , "docs":[{"doc":docIndex , "indexes":[i]}]})] # word is bigger that all other words => we can append it to end of the list

            else :
                postingList.append({"word":word , "docs":[{"doc":docIndex , "indexes":[i]}]}) # we have not already added the word and dou to the fact that they are read in order of their index, we can easily add append new one in end of the list



    def input (self, filePath: list[str]): # input paths of inputs
        tempPostingList=[]
        if(os.path.exists(self.tempDirectory)):
            shutil.rmtree(self.tempDirectory)
        os.mkdir(self.tempDirectory)
        saved=False
        for i in range(len(filePath)): # for files in input
            
            if i%5==0:
                if (i>0):
                    # self.subPostingLists.append(copy.deepcopy(tempPostingList))
                    # print(tempPostingList)
                    saved=True
                    with open(f'{self.tempDirectory}/saved_postingList{int(i/5)-1}.pkl', 'wb') as f: # save subPostingList on a file
                        pickle.dump(tempPostingList, f)
                tempPostingList=[]
            saved = False
            file = open(filePath[i],'r',encoding='cp1252') # open the file
            text = file.read() # read the file
            file.close()  # close the file
            # tokenize text and ignore stopping words using nltk library 
            tokenizedText = [word for word in word_tokenize(text.lower(),preserve_line=False) if word not in self.stop] 
            print (f'document {i+1} : {filePath[i]}')
            # print(tokenizedText)
            self.addToPostingList(tokenizedText , i+1 ,tempPostingList) # i indicates to index of document we are reading
        if (not saved):
            # self.subPostingLists.append(copy.deepcopy(tempPostingList))
            with open(f'{self.tempDirectory}/saved_postingList{int((len(filePath)-1)/5)}.pkl', 'wb') as f:
                pickle.dump(tempPostingList, f)
            tempPostingList=[]
        

    def numberToGamaCode(self, n):  # new function for homework 3
        binaryNumber = bin(n)[2:]
        ans = ""
        for _ in range (len(binaryNumber)):
            ans+="1"
        ans+="0"+binaryNumber[1:]
        return ans



    def listToGamaCode(self, ls , startingIndex = 0):  # new function for homework 3
        temp = ""
        s = startingIndex
        for item in ls:
            temp+=self.numberToGamaCode(item-s)
            s = item
        return temp
    
    def gamaCodeToNumber(self,gamaCode):
        oneCounter=0
        for i in gamaCode:
            if i == '1':
                oneCounter+=1
            else: break
        if oneCounter==1:return 1
        return int("1"+gamaCode[oneCounter+1:],2)
        

    
    def gamaCodeToList(self,gamaCode):
        temp = []
        s=0
        e=0
        while(s!=len(gamaCode)):
            if gamaCode[e] =="1":
                e+=1
            else :
                e=(e-s)*2+s
                temp.append(gamaCode[s:e])
                s=e
        
        ls =  list(map(self.gamaCodeToNumber,temp))
        for i in range(1,len(ls)):
            ls[i]+=ls[i-1]
        return ls



    def mergePostingLists (self): # new function for homework 3
        codedPostingList = self.codedPostingLit
        ls = sorted(os.listdir(self.tempDirectory))

        for file in ls:
            with open(f'{self.tempDirectory}/{file}', 'rb') as f:
                subPostingList=pickle.load( f)
            for item in subPostingList:
                word = item["word"]
                index = self.searchPostingList(word,codedPostingList) # find index of word in posting list
                if (len(codedPostingList)>index): # check if index is not larger than posting list (if word is bigger that all words, search function returns len(postingList)+1)
                    if (codedPostingList[index]["word"] == word): # check if index is the index of the word
                        codedPostingList[index]["docs"]+=self.listToGamaCode([document["doc"] for document in item["docs"] ],codedPostingList[index]["lastSeenIndex"])
                        codedPostingList["lastSeenIndex"]:item["docs"][-1]["doc"]

                    else :
                        codedPostingList[index:index]=[{"word":item["word"],"docs": self.listToGamaCode([document["doc"] for document in item["docs"] ]) , "lastSeenIndex":item["docs"][-1]["doc"]}]

                else :
                    codedPostingList.append({"word":item["word"],"docs": self.listToGamaCode([document["doc"] for document in item["docs"] ]) , "lastSeenIndex":item["docs"][-1]["doc"]})
            # print("gama is :")
            # print(self.codedPostingLit)


    def findWord(self, word): # this function use our binary search function to find word in posting list and if the word is not included in the list, returns -1
        index = self.searchPostingList(word,self.codedPostingLit)
        if (index< len(self.codedPostingLit)):
            if (self.codedPostingLit[index]["word"] == word):
                return index # real index of the word
        return -1 # word is not in the posting list


    def find(self , query:str): # split the query and find the result 
        splitQuery = query.lower().split()
        if (len(splitQuery)==1): # query is only one word
            index = self.findWord(splitQuery[0])
            if (index>-1):
                print("word appears in :  ", self.codedPostingLit[index]["docs"])
                return [ i for i in self.gamaCodeToList(self.codedPostingLit[index]["docs"])]
            return []

        else: 
            index1 = self.findWord(splitQuery[0])
            index2 = self.findWord(splitQuery[2])
            if splitQuery[1] in ["and" ,"or", "AND", "OR"]: # boolean condition
                
                if (splitQuery[1] in ["and","AND"]): # and condition
                    if(index1!=-1 and index2!=-1): # check if there are result for both of words
                        print("first word is in :  ", self.codedPostingLit[index1]["docs"])
                        print("second word is in :  ", self.codedPostingLit[index2]["docs"])

                        docs1 =  set([ i for i in self.gamaCodeToList(self.codedPostingLit[index1]["docs"])]) # find document of word one 
                        docs2 =  set([ i for i in self.gamaCodeToList(self.codedPostingLit[index2]["docs"])]) # find document of word two 
                        # print(self.codedPostingLit[index1]["docs"] , self.codedPostingLit[index2]["docs"])
                        return list(docs1.intersection(docs2)) # make intersection of two results
                    return []
                    
                if (splitQuery[1] in ["or","OR"]): # or condition
                    if(index1!=-1 and index2!=-1): # check if there are result for both of words
                        print("first word is in :  ", self.codedPostingLit[index1]["docs"])
                        print("second word is in :  ", self.codedPostingLit[index2]["docs"])
                        docs1 =  set([ i for i in self.gamaCodeToList(self.codedPostingLit[index1]["docs"])])
                        docs2 =  set([ i for i in self.gamaCodeToList(self.codedPostingLit[index2]["docs"])])
                        # print(self.codedPostingLit[index1]["docs"] , self.codedPostingLit[index2]["docs"])
                        return list(docs1.union(docs2))
                    print("first word is in :  ", self.codedPostingLit[index1]["docs"])
                    print("second word is in :  ", self.codedPostingLit[index2]["docs"])
                    if(index1!=-1): return [ i for i in self.gamaCodeToList(self.codedPostingLit[index1]["docs"])] # check if there is result for first word
                    return [ i for i in self.gamaCodeToList(self.codedPostingLit[index2]["docs"])] # check if there is result for second word

            # else : # near condition
            #     nearNumber = int(splitQuery[1].split('/')[1]) # find near 
            #     if(index1!=-1 and index2!=-1): # check if there are result for both of words
            #         docs1 =   {i["doc"]:i["indexes"] for i in self.postingList[index1]["docs"]} # find document of word one and make dictionary for result
            #         docs2 =   {i["doc"]:i["indexes"] for i in self.postingList[index2]["docs"]} # find document of word two and make dictionary for result

            #         result = [] 
            #         keysOfDocs2 = docs2.keys()
            #         for key, value in docs1.items():
            #             if (key in keysOfDocs2):
            #                 d1 = np.array(value) # indexes of word in first document
            #                 d2 = np.array(copy.deepcopy(docs2[key])) # indexes of word in second document
            #                 distanceMatrix = np.array([[abs(i - j )for i in d1] for j in d2]) # distance matrix
            #                 if (distanceMatrix.min() <= nearNumber): # check validation
            #                     result.append(key)


            #                 # example of distanceMatrix
            #                 # 
            #                 # [[ 0  1  2 10 11]
            #                 # [ 1  0  1  9 10]
            #                 # [ 2  1  0  8  9]
            #                 # [10  9  8  0  1]
            #                 # [11 10  9  1  0]]


                    return result
            return []
            


    def prnt(self):
        for i in self.postingList:
            print(i)

    def printGamaCodePostingList(self):
        print("gama code posting list is : ")
        for i in self.codedPostingLit:
            print(i)

    def printPostingListsFiles(self):
        ls = sorted(os.listdir(self.tempDirectory))
        print(ls)
        for file in ls:
            with open(f'{self.tempDirectory}/{file}', 'rb') as f:
                print(pickle.load( f))




# Test cases:

In [1277]:
import os
directory_input = ['docs/'+path for path in os.listdir('docs') if path[-3:] == 'txt']
print(directory_input)

['docs/Jerry Decided To Buy a Gun.txt', 'docs/Rentals at the Oceanside Community.txt', 'docs/Gasoline Prices Hit Record High.txt', 'docs/Cloning Pets.txt', 'docs/Crazy Housing Prices.txt', 'docs/Man Injured at Fast Food Place.txt', 'docs/A Festival of Books.txt', 'docs/Food Fight Erupted in Prison.txt', 'docs/Better To Be Unlucky.txt', 'docs/Sara Went Shopping.txt', 'docs/Freeway Chase Ends at Newsstand.txt', 'docs/Trees Are a Threat.txt', 'docs/A Murder-Suicide.txt', 'docs/Happy and Unhappy Renters.txt', 'docs/Pulling Out Nine Tons of Trash.txt']


In [1278]:
test_directory = searchEngine(tempDirectory="test_directory_postingLists")
test_directory.input(directory_input)
test_directory.mergePostingLists()

document 1 : docs/Jerry Decided To Buy a Gun.txt
document 2 : docs/Rentals at the Oceanside Community.txt
document 3 : docs/Gasoline Prices Hit Record High.txt
document 4 : docs/Cloning Pets.txt
document 5 : docs/Crazy Housing Prices.txt
document 6 : docs/Man Injured at Fast Food Place.txt
document 7 : docs/A Festival of Books.txt
document 8 : docs/Food Fight Erupted in Prison.txt
document 9 : docs/Better To Be Unlucky.txt
document 10 : docs/Sara Went Shopping.txt
document 11 : docs/Freeway Chase Ends at Newsstand.txt
document 12 : docs/Trees Are a Threat.txt
document 13 : docs/A Murder-Suicide.txt
document 14 : docs/Happy and Unhappy Renters.txt
document 15 : docs/Pulling Out Nine Tons of Trash.txt


In [1279]:
test = searchEngine(tempDirectory="test_postingLists")

test.input(['document1.txt','document2.txt','document3.txt'])
test.mergePostingLists()


document 1 : document1.txt
document 2 : document2.txt
document 3 : document3.txt


In [1280]:
test.printGamaCodePostingList()

gama code posting list is : 
{'word': 'another', 'docs': '110010', 'lastSeenIndex': 3}
{'word': 'boolean', 'docs': '1101', 'lastSeenIndex': 3}
{'word': 'capabilities', 'docs': '1101', 'lastSeenIndex': 3}
{'word': 'contains', 'docs': '101100', 'lastSeenIndex': 3}
{'word': 'content', 'docs': '110010', 'lastSeenIndex': 3}
{'word': 'different', 'docs': '1100', 'lastSeenIndex': 2}
{'word': 'document', 'docs': '101010', 'lastSeenIndex': 3}
{'word': 'example', 'docs': '101010', 'lastSeenIndex': 3}
{'word': 'important', 'docs': '1100', 'lastSeenIndex': 2}
{'word': 'indexed', 'docs': '10', 'lastSeenIndex': 1}
{'word': 'indexing', 'docs': '1100', 'lastSeenIndex': 2}
{'word': 'processed', 'docs': '10', 'lastSeenIndex': 1}
{'word': 'relevant', 'docs': '1101', 'lastSeenIndex': 3}
{'word': 'retrieval', 'docs': '1100', 'lastSeenIndex': 2}
{'word': 'search', 'docs': '1101', 'lastSeenIndex': 3}
{'word': 'several', 'docs': '10', 'lastSeenIndex': 1}
{'word': 'simple', 'docs': '10', 'lastSeenIndex': 1}
{'

In [1281]:
test_directory.printGamaCodePostingList()

gama code posting list is : 
{'word': '.38', 'docs': '10', 'lastSeenIndex': 1}
{'word': '1', 'docs': '1101', 'lastSeenIndex': 3}
{'word': '1,000', 'docs': '1011110011', 'lastSeenIndex': 1}
{'word': '10', 'docs': '101010110011001101', 'lastSeenIndex': 5}
{'word': '1000', 'docs': '11110001', 'lastSeenIndex': 9}
{'word': '1000x', 'docs': '11110001', 'lastSeenIndex': 9}
{'word': '100x', 'docs': '11110001', 'lastSeenIndex': 9}
{'word': '10x', 'docs': '11110001', 'lastSeenIndex': 9}
{'word': '11', 'docs': '11110100', 'lastSeenIndex': 12}
{'word': '110,000', 'docs': '11110001', 'lastSeenIndex': 9}
{'word': '12', 'docs': '1101', 'lastSeenIndex': 3}
{'word': '120', 'docs': '11110001', 'lastSeenIndex': 9}
{'word': '120,000', 'docs': '11110001', 'lastSeenIndex': 9}
{'word': '15', 'docs': '1100', 'lastSeenIndex': 2}
{'word': '15-percent', 'docs': '1100', 'lastSeenIndex': 2}
{'word': '150', 'docs': '11101111110000', 'lastSeenIndex': 7}
{'word': '1992.', 'docs': '11110010', 'lastSeenIndex': 10}
{'wo

In [1282]:
test.printPostingListsFiles()

['saved_postingList0.pkl']
[{'word': 'another', 'docs': [{'doc': 2, 'indexes': [0]}, {'doc': 3, 'indexes': [0]}]}, {'word': 'boolean', 'docs': [{'doc': 3, 'indexes': [4]}]}, {'word': 'capabilities', 'docs': [{'doc': 3, 'indexes': [6]}]}, {'word': 'contains', 'docs': [{'doc': 1, 'indexes': [3]}, {'doc': 3, 'indexes': [8]}]}, {'word': 'content', 'docs': [{'doc': 2, 'indexes': [4]}, {'doc': 3, 'indexes': [10]}]}, {'word': 'different', 'docs': [{'doc': 2, 'indexes': [3]}]}, {'word': 'document', 'docs': [{'doc': 1, 'indexes': [2]}, {'doc': 2, 'indexes': [2, 5]}, {'doc': 3, 'indexes': [2, 7]}]}, {'word': 'example', 'docs': [{'doc': 1, 'indexes': [1]}, {'doc': 2, 'indexes': [1]}, {'doc': 3, 'indexes': [1]}]}, {'word': 'important', 'docs': [{'doc': 2, 'indexes': [7]}]}, {'word': 'indexed', 'docs': [{'doc': 1, 'indexes': [8]}]}, {'word': 'indexing', 'docs': [{'doc': 2, 'indexes': [6]}]}, {'word': 'processed', 'docs': [{'doc': 1, 'indexes': [7]}]}, {'word': 'relevant', 'docs': [{'doc': 3, 'index

In [1283]:
test_directory.printPostingListsFiles()

['saved_postingList0.pkl', 'saved_postingList1.pkl', 'saved_postingList2.pkl']
[{'word': '.38', 'docs': [{'doc': 1, 'indexes': [103]}]}, {'word': '1', 'docs': [{'doc': 3, 'indexes': [120]}]}, {'word': '1,000', 'docs': [{'doc': 1, 'indexes': [47]}]}, {'word': '10', 'docs': [{'doc': 1, 'indexes': [122]}, {'doc': 2, 'indexes': [51]}, {'doc': 3, 'indexes': [58]}, {'doc': 5, 'indexes': [81]}]}, {'word': '12', 'docs': [{'doc': 3, 'indexes': [21]}]}, {'word': '15', 'docs': [{'doc': 2, 'indexes': [160]}]}, {'word': '15-percent', 'docs': [{'doc': 2, 'indexes': [22]}]}, {'word': '1993', 'docs': [{'doc': 5, 'indexes': [42]}]}, {'word': '2,000', 'docs': [{'doc': 2, 'indexes': [38]}]}, {'word': '2.09', 'docs': [{'doc': 3, 'indexes': [29]}]}, {'word': '2.14', 'docs': [{'doc': 3, 'indexes': [80]}]}, {'word': '2.22', 'docs': [{'doc': 3, 'indexes': [15]}]}, {'word': '20', 'docs': [{'doc': 3, 'indexes': [59, 102]}]}, {'word': '20,000', 'docs': [{'doc': 5, 'indexes': [102]}]}, {'word': '200', 'docs': [{'

# Test Query:

In [1284]:
test.find("contains")

word appears in :   101100


[1, 3]

In [1285]:
test_directory.find("several")

word appears in :   110111110000


[3, 11]

In [1286]:
test_directory.find("several and 10")


first word is in :   110111110000
second word is in :   101010110011001101


[3]

In [1287]:
test_directory.find("several or 10")


first word is in :   110111110000
second word is in :   101010110011001101


[1, 2, 3, 5, 7, 10, 11]